# aMapReduce Framework

Agentics enable scalable execution of LLM workflows by implementing a MapReduce framework which enable the async use of LLM blended with regular python code.

In [7]:
! uv pip install agentics-py


import os
from pathlib import Path
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive
    drive.mount("/content/drive")
    load_dotenv("/content/drive/MyDrive/.env")
else:
    
    CURRENT_PATH=os.getcwd()
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

base = Path(CURRENT_PATH)

Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics911/agentics/.venv
Audited 1 package in 20ms
In Colab: False


Let us first define an aType to represent StockMarket Data for the DowJones index, and populate it with historical data

In [19]:
from agentics import Agentics as AG
from typing import Optional
from pydantic import BaseModel, Field
from datetime import datetime
## Define the data model for stock market data

class StockMarketState(BaseModel):
    date: Optional[str] = None
    open: Optional[float] = None
    high: Optional[float] = None
    low: Optional[float] = None
    close: Optional[float] = None
    volume: Optional[float] = None
    daily_range: Optional[float] = Field(None, 
        description="""The difference between the high and low prices for the day.""")
    news: Optional[str] = Field(None, 
        description="""Text reporting a list of news headlines relevant to the stock for the day.""")
    explanation_report: Optional[str] = Field(None,
        description="A detailed explanation of the stock market state for the day.")

    
## import the data
dj_data = AG.from_csv(base / "data/dow_jones.csv", atype=StockMarketState)
for state in dj_data[:3]: 
    print(state)

2025-09-12 11:47:40.320 | DEBUG    | agentics.core.agentics:from_csv:312 - Importing Agentics of type StockMarketState from CSV /Users/gliozzo/Code/agentics911/agentics/tutorials/data/dow_jones.csv


date='2016-07-01' open=17924.240234 high=18002.380859 low=17916.910156 close=17949.369141 volume=82160000.0 daily_range=None news=None explanation_report=None
date='2016-06-30' open=17712.759766 high=17930.609375 low=17711.800781 close=17929.990234 volume=133030000.0 daily_range=None news=None explanation_report=None
date='2016-06-29' open=17456.019531 high=17704.509766 low=17456.019531 close=17694.679688 volume=106380000.0 daily_range=None news=None explanation_report=None


## Amap

Amap functions enable async execution of functions over all the states of an AG. Agentics supports 1:1 maps that maps all states of an AG into states of the same type.

In the following example we define a simple function to compute the daily_range of the stock and we pass that to an amap fuction which applies that to all states asyncronously

In [20]:
## Note that input and output are both StockMarketState objects
async def get_daily_variation_percentage(state: StockMarketState) -> StockMarketState:
    state.daily_range = (float(state.high) - float(state.low)) / float(state.low) * 100
    return state

## Apply the function to all states using amap
dj_data.batch_size = 100
dj_data = await dj_data.amap(get_daily_variation_percentage)

for state in dj_data[:3]: 
    print(f"Date: {state.date}, Daily Range: {state.daily_range}")

2025-09-12 11:48:26.429 | DEBUG    | agentics.core.agentics:amap:206 - Executing amap on function <function get_daily_variation_percentage at 0x15a4c1120>
2025-09-12 11:48:26.431 | DEBUG    | agentics.core.agentics:amap:231 - 100 states processed. 9.920597076416015e-06 seconds average per state in the last chunk ...
2025-09-12 11:48:26.432 | DEBUG    | agentics.core.agentics:amap:231 - 200 states processed. 1.1799335479736329e-05 seconds average per state in the last chunk ...
2025-09-12 11:48:26.434 | DEBUG    | agentics.core.agentics:amap:231 - 300 states processed. 1.0890960693359375e-05 seconds average per state in the last chunk ...
2025-09-12 11:48:26.435 | DEBUG    | agentics.core.agentics:amap:231 - 400 states processed. 7.982254028320312e-06 seconds average per state in the last chunk ...
2025-09-12 11:48:26.436 | DEBUG    | agentics.core.agentics:amap:231 - 500 states processed. 6.608963012695312e-06 seconds average per state in the last chunk ...
2025-09-12 11:48:26.437 | DE

Date: 2016-07-01, Daily Range: 0.47703930117312493
Date: 2016-06-30, Daily Range: 1.2353831025172834
Date: 2016-06-29, Daily Range: 1.423521751672572


## aReduce

Reduce functions enable executing operations on the entire list of elements (states) within an Agentics group. Although reduce operations are intrinsically synchronous—since they consider all states at once—they are defined as async functions to allow for internal async calls (such as fetching news or running LLMs).

In the following example we will use a reduce function to analyze get the top 10 days with highest variation in the market

In [21]:
async def get_highest_volatility_days(states:list[StockMarketState]) -> list[StockMarketState]:
    
    # sort the states by volatility and return the top 10, define a new AG with these states
    return sorted(states, 
                key=lambda x: abs(x.daily_range) if x.daily_range is not None else 0, 
                reverse=True)[:10]

# apply the reduce function to get the top 10 days with highest volatility
highest_volatility_days = await dj_data.areduce(get_highest_volatility_days)
print(highest_volatility_days.pretty_print())

Atype : <class '__main__.StockMarketState'>
date: '2008-10-10'
open: 8568.669922
high: 8901.280273
low: 7882.509766
close: 8451.19043
volume: 674920000.0
daily_range: 12.924443321266926
news: null
explanation_report: null

date: '2008-11-13'
open: 8281.139648
high: 8876.589844
low: 7965.419922
close: 8835.25
volume: 476600000.0
daily_range: 11.439069514507388
news: null
explanation_report: null

date: '2008-10-13'
open: 8462.419922
high: 9427.990234
low: 8462.179688
close: 9387.610352
volume: 399290000.0
daily_range: 11.413259722782675
news: null
explanation_report: null

date: '2008-10-28'
open: 8178.720215
high: 9082.080078
low: 8174.72998
close: 9065.120117
volume: 372160000.0
daily_range: 11.099450382090795
news: null
explanation_report: null

date: '2010-05-06'
open: 10868.120117
high: 10879.759766
low: 9869.620117
close: 10520.320312
volume: 459890000.0
daily_range: 10.23483819058118
news: null
explanation_report: null

date: '2008-10-09'
open: 9261.69043
high: 9448.139648
low: 8

## Complex AMAPs

aMaps function can contain external API and LLM calls. This way we can use agentics as a scaleout frameworks for complex workflows. 

In [22]:
from ddgs import DDGS

## Define a function to get news for a given date using the DDGS search engine
## Note that the similar functionalities can be implemented using MCP tools in AGs
async def get_news(state):
    state.news=str(DDGS().text(f"What happended to the stock market and dow jones on {state.date}", max_results=10))
    return state    

## set the batch size for the amap function to 5 (only 10 states will be processed)
highest_volatility_days.batch_size = 10

# Now get news for the top 10 days with highest volatility using amap
highest_volatility_days = await highest_volatility_days.amap(get_news)

# print the first result for brevity
print(highest_volatility_days[0])
#print(f"Date: {highest_volatility_days[0].date}, Daily Range: {highest_volatility_days[0].daily_range}, News: {highest_volatility_days[0].news[:200]}")

2025-09-12 11:49:28.549 | DEBUG    | agentics.core.agentics:amap:206 - Executing amap on function <function get_news at 0x15a4c31a0>
2025-09-12 11:49:44.109 | DEBUG    | agentics.core.agentics:amap:231 - 10 states processed. 1.5559696912765504 seconds average per state in the last chunk ...


date='2008-10-10' open=8568.669922 high=8901.280273 low=7882.509766 close=8451.19043 volume=674920000.0 daily_range=12.924443321266926 news='[{\'title\': \'Wikipedia Stock market crash - Wikipedia\', \'href\': \'https://en.wikipedia.org/wiki/Stock_market_crash\', \'body\': \'5 days ago - From October 6–10, 2008, the Dow Jones Industrial Average (DJIA) closed lower in all five sessions . Volume levels were record-breaking. The DJIA fell over 1,874 points, or 18%, in its worst weekly decline ever on both a points and percentage basis.\'}, {\'title\': \'Wikipedia United States bear market of 2007–2009 - Wikipedia\', \'href\': \'https://en.wikipedia.org/wiki/United_States_bear_market_of_2007–2009\', \'body\': \'August 4, 2025 - The US bear market of 2007–2009 was a bear market that lasted from October 9, 2007 to March 9, 2009, encompassing the 2008 financial crisis. The S&P 500 lost approximately 50% of its value , but the duration of the bear market was just below average. The bear market

Now let's use self transduction to provide an explanation for the market volatility

In [23]:
from agentics.core.llm_connections import get_llm_provider
highest_volatility_days.instructions = """Explain the reasons why the market went down or up 
given the high volatility in the stock market on this day based on the news provided. 
Provide a concise summary."""
highest_volatility_days.llm= get_llm_provider() ## You can choose between "openai", "watsonx", "gemini", "vllm_crewai"
highest_volatility_explanations = await highest_volatility_days.self_transduction(
["date", "open", "high", "low", "close", "volume", "daily_range", "news"],["explanation_report"])

for state in highest_volatility_explanations: 
    print(f"Date: {state.date}, Daily Range: {state.daily_range}\nExplanation: {state.explanation_report}...")

2025-09-12 11:51:18.682 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-12 11:51:18.683 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'
2025-09-12 11:51:18.684 | DEBUG    | agentics.core.agentics:__lshift__:518 - Executing task: Explain the reasons why the market went down or up 
given the high volatility in the stock market on this day based on the news provided. 
Provide a concise summary.
10 states will be transduced
2025-09-12 11:51:18.685 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-12 11:51:39.754 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 10 states in 21.06796383857727 seconds
2025-09-12 11:51:39.754 | DEBUG    | agentics.core.agentics:__lshift__:700 - 10 states processed in

Date: 2008-10-10, Daily Range: 12.924443321266926
Explanation: The market went down on this day due to the high volatility in the stock market, which was caused by the 2008 financial crisis. The Dow Jones Industrial Average (DJIA) closed lower in all five sessions from October 6-10, 2008, with a record-breaking decline of 1,874 points, or 18.2%, in its worst weekly decline ever on both a points and percentage basis. This was a result of the global financial crisis, which was triggered by the subprime mortgage crisis and led to a freeze in credit markets, causing sellers to shed their stocks. The crisis led to a significant loss in market value, with the Dow Jones industrial average falling nearly 778 points, or 6.98%, to 10,365.45 on September 29, 2008. The market's decline was further exacerbated by Congress's inability to pass the Bush administration's bill, which led to fears that the credit markets wouldn't be able to rebound quickly. The stock market crash of 2008 had a significan

## Well Done
You are now fully equipped to work with agentics and apply it to your data.
Congratulations and please contribute back to the community if you feel this is exciting. 